In [163]:
import os
import requests
import json
import time
import datetime
from bs4 import BeautifulSoup


Le Configurations de base du projet:
la langue pour laquelle on souhaite reccuperer la bible ainsique les differents livres de la bible qui existe

_gen1_ represente le premier livre de la Genèse
C'est le livre par defaut du site


Commençons par chercher tous les id de livres, pour cela on cherche juste les valuers des attributs _`data-vars-event-label`_ des elements selectionnés par _`ul.list.ma0.pa0.bg-white.pb5.min-vh-100 li`_ ceci sera fait grace à la console du navigateur puis on mettra le resultat dans un fichier json _utils/livres.json_


In [166]:
# Langue dans laquelle on va travailler
lang = 3307  # bible en Haussa avec caractères type arabique

VERSE_SELECTOR = ".yv-gray50.lh-copy.f3-m"

# dossier dans lequel on va mettre les dossiers chaque chapitre pour cette langue


def get_lang_folder_path(lang: int = lang) -> str:
    path = os.path.join(os.getcwd(), f"/data/{lang}")
    if not os.path.exists(path):
        os.makedirs(path)
    return path

# dossier dans lequel on va mettre les fichiers txt pour chaqe chap du livre livre


def get_book_folder_path_for_lang(book: str, lang: int = lang) -> str:
    path = os.path.join(os.getcwd(), f"data/{lang}/{book}").replace('\\', '/')

    if not os.path.exists(path):
        print(f'Path to saves folder created : {path} ')
        os.makedirs(path)
    return path


# chargeons les livres
with open('utils/livres.json', 'r') as books_file:
    books = json.load(books_file)

gen1 = requests.get(f"https://www.bible.com/bible/{lang}/GEN.1.NFC")


def chapter_link(chapter: str, lang: int = lang) -> str:
    return f"https://www.bible.com/bible/{lang}/{chapter}"


def verse_link(chapter: str, verse: int, lang: int = lang) -> str:
    return f"https://www.bible.com/bible/{lang}/{chapter}.{verse}"


Pour trouver le nombre de chapitres d'un livre, nous ferons des requettes et à chaque en incrementant un compteur jusqu'à ce qu'il y ai redirection vers la Genèse 1 sachat que le nombre de chapitre varie entre 1 et 150


In [ ]:
# les urls des json des chapitres pour la langue en cours
chapters_url = f"https://www.bible.com/json/bible/books/{lang}"
chapters: dict[list] = {}

not_founds = 0
for book in books:

    res = requests.get(f"{chapters_url}/{book}/chapters")
    if res.status_code == 500:
        print(f"[{book}] Not Found ..")
        not_founds += 1
        continue
    chapters[book] = []
    data = json.loads(res.content)
    for item in data['items']:
        chapters[book].append(item['usfm'])

    print(f"[{book}] Completed ..")

print(
    f"\n\n\n[{lang}] Chapters fetched with success, {not_founds} books not found")
chapters


Sauvegarder les de chapitres par langue dans chaque langue aura un fichier json dans lequel on mettre un dicco.
Le dossier `./utils/chapitres_id/` contiendra des fichiers json pour chaque langue, chaque fichier contiendra tous les id de chapitres rangés par livre


In [161]:
chapters_id_file = os.path.join(
    os.getcwd(), f'utils/chapitres_id').replace('\\', '/')
if not os.path.exists(chapters_id_file):
    os.makedirs(chapters_id_file)
with open(f'{chapters_id_file}/{lang}.json', 'w', encoding='utf-8') as lang_chaps_file:
    json.dump(chapters, lang_chaps_file)


Etant donné qu'on connait les id de chaque chapitre pour la langue, on peut commencer à chercher les versets de chaque chapitre


Enregistrer les versets dans un fichier txt

In [167]:
done_books = []
for book_name in chapters:
    book = chapters[book_name]
    print(f'[{book_name}] Proceeding book ...')
    for chapter in book[:2]:
        print(f'[{chapter}] Proceeding chapter...')
        # selectionner un chapitre ainsi que sa page/url
        chapter_res = requests.get(chapter_link(chapter))
        can_start_verses = chapter_res.status_code == 200
        while not can_start_verses:
            print('Internet Connexion error, We will retry in 5 secs')
            time.sleep(5)
            chapter_res = requests.get(chapter_link(chapter))
            can_start_verses = chapter_res.status_code == 200
        # les versets du chapitre
        chapter_verses: list[str] = []
        
        
        verse_index = 1
        # commencer à cchercher les versets duc chapitre du livre dans lequel on est
        while True:
            link = verse_link(chapter, verse_index)
            verse_res = requests.get(link)
            if verse_res.status_code != 200:
                continue
            # si le verset n'existe pas on sera redirigé vers la page du chapitre
            soup = BeautifulSoup(verse_res.content, 'html.parser')
            verses = soup.select(VERSE_SELECTOR)
            if verse_res.url == chapter_res.url or verse_res.url == gen1.url or len(verses) == 0:
                break
            
            chapter_verses.append(verses[0].text.replace('\n', ' '))
            print(f"[{chapter}][{verse_index}] Verse fetched with success")
            verse_index += 1

        folder = get_book_folder_path_for_lang(book_name)

        with open(f'{folder}/{chapter}.txt', mode='w', encoding='utf-8') as verse_output_file:
            for verse in chapter_verses:
                verse.replace('\n', ' ')
                verse_output_file.write(verse.strip() + '\n')

        with open('rescue.txt', 'w', encoding='utf-8') as save:
            save.write(
                f'Last saves\nlanguage:{lang}\nCurrent book:{book_name} --> {"Completed" if book.index(chapter) == len(book) else "Not Completed"}\nChapter:{chapter}\nDone books:{done_books}\nChapters done in the last/current book: {book[:book.index(chapter)+1]}')

        print(f'[{chapter}] done')

    done_books.append(book_name)
    print(f'[{book_name}] done')


[GEN] Proceeding book ...
[GEN.1] Proceeding chapter...
[GEN.1][1] Verse fetched with success
[GEN.1][2] Verse fetched with success
[GEN.1][3] Verse fetched with success
[GEN.1][4] Verse fetched with success
[GEN.1][5] Verse fetched with success
[GEN.1][6] Verse fetched with success
[GEN.1][7] Verse fetched with success
[GEN.1][8] Verse fetched with success
[GEN.1][9] Verse fetched with success
[GEN.1][10] Verse fetched with success
[GEN.1][11] Verse fetched with success
[GEN.1][12] Verse fetched with success
[GEN.1][13] Verse fetched with success
[GEN.1][14] Verse fetched with success
[GEN.1][15] Verse fetched with success
[GEN.1][16] Verse fetched with success
[GEN.1][17] Verse fetched with success
[GEN.1][18] Verse fetched with success
[GEN.1][19] Verse fetched with success
[GEN.1][20] Verse fetched with success
[GEN.1][21] Verse fetched with success
[GEN.1][22] Verse fetched with success
[GEN.1][23] Verse fetched with success
[GEN.1][24] Verse fetched with success
[GEN.1][25] Verse

KeyboardInterrupt: 

In [165]:
now = datetime.datetime.now()
time.sleep(5)
datetime.datetime.now() - now


datetime.timedelta(seconds=5, microseconds=8743)